# Fake News Detector using LSTM

Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string


Fetching data

In [2]:
data_fake=pd.read_csv("Fake news detector/Fake.csv")
data_true=pd.read_csv("Fake news detector/True.csv")

In [3]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
data_fake["class"]=0
data_true["class"]=1

In [6]:
data_fake.shape, data_true.shape

((23481, 5), (21417, 5))

In [7]:
data_true.head()

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [8]:
data_merge=pd.concat([data_fake,data_true],axis=0)

In [9]:
data_merge.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [10]:
data=data_merge.drop(['text','subject','date'],axis=1)

In [11]:
data.head()

,title,class
0,Donald Trump Sends Out Embarrassing New Year’...,0
1,Drunk Bragging Trump Staffer Started Russian ...,0
2,Sheriff David Clarke Becomes An Internet Joke...,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,0
4,Pope Francis Just Called Out Donald Trump Dur...,0


In [12]:
data.isnull().sum()

title    0
class    0
dtype: int64

In [13]:
data=data.sample(frac=1)
data.head()

,title,class
5021,WATCH: 5 Straight Minutes Of Donald Trump Lyi...,0
5696,Fox News Attacks LGBT Community With Disgusti...,0
741,Trump’s Lawyers Say Encouraging Reasonable Fo...,0
11161,U.S. lawmakers delay bill on European data pri...,1
3663,The Media TOTALLY Missed This Horrifying Trum...,0


In [14]:
data.reset_index(inplace=True)
data.drop(["index"],axis=1,inplace=True)

In [15]:
data.columns

Index(['title', 'class'], dtype='object')

In [16]:
data.head()

,title,class
0,WATCH: 5 Straight Minutes Of Donald Trump Lyi...,0
1,Fox News Attacks LGBT Community With Disgusti...,0
2,Trump’s Lawyers Say Encouraging Reasonable Fo...,0
3,U.S. lawmakers delay bill on European data pri...,1
4,The Media TOTALLY Missed This Horrifying Trum...,0


In [17]:
## Get the Independent Features

X=data.drop('class',axis=1)

## Get the Dependent features
y=data['class']

In [18]:
X.shape, y.shape

((44898, 1), (44898,))

In [19]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [20]:
messages=X.copy()

In [21]:
messages['title'][1]

' Fox News Attacks LGBT Community With Disgusting Story Saying They’re After Young Children'

In [22]:
messages

,title
0,WATCH: 5 Straight Minutes Of Donald Trump Lyi...
1,Fox News Attacks LGBT Community With Disgusti...
2,Trump’s Lawyers Say Encouraging Reasonable Fo...
3,U.S. lawmakers delay bill on European data pri...
4,The Media TOTALLY Missed This Horrifying Trum...
...,...
44893,Kanye West appears to dump Trump with disappea...
44894,AMERICAN UNIVERSITY HIRES Former Islamic Terro...
44895,LOL! OBAMA’S RADICAL EPA CHIEF Says There Was ...
44896,"Germany softens stance on Turkish arms sales, ..."


Data Preprocessing from here

In [23]:
import nltk
import re
from nltk.corpus import stopwords

In [24]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Stemming

In [25]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [26]:
corpus[1]

'fox news attack lgbt commun disgust stori say young children'

In [27]:
### Vocabulary size
voc_size=5000

One Hot Encoding

In [28]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3422, 1025, 474, 2104, 1539, 3976, 908],
 [3519, 2843, 3717, 441, 4185, 332, 4091, 3438, 835, 3828],
 [1539, 1338, 3438, 3148, 3757, 4964, 1824, 4883, 2499, 4795],
 [1266, 3054, 405, 2847, 1644, 3894, 3638, 1237],
 [3461, 951, 1420, 3437, 1539, 4591, 2006],
 [1539, 3438, 954, 3818, 1945, 3309, 3818, 1945, 2006],
 [4448, 4832, 337, 928, 1368, 4947, 1857, 2385, 75, 2936],
 [898, 4646, 2139, 3244, 3673, 3367, 4336, 3945],
 [4428, 29, 2678, 3438, 3645, 4327, 482, 230, 2548, 4860],
 [4227, 3291, 2617, 4656, 1980, 4944, 3031, 3896],
 [4907, 1317, 2766, 4947, 206, 4336, 1539, 1941, 2009],
 [3182, 517, 2113, 4572, 337, 2851, 898],
 [270, 282, 3849, 1588, 2437, 1275, 4104, 2469, 576],
 [2726, 3275, 2698, 3437, 1010, 1266, 2401, 2719],
 [1539, 2456, 4819, 1530, 4444, 1266, 2731],
 [3378, 2685, 3127, 102, 4482, 2923, 3273, 2467, 1734, 4537, 2669, 4746, 2745],
 [1856, 1587, 3438, 4336, 1266, 2456, 2818, 4656, 792, 2005],
 [3511, 3466, 1317, 2711, 1734, 1539],
 [804, 483, 3290, 1995, 3688, 915, 4

In [29]:
corpus[1]

'fox news attack lgbt commun disgust stori say young children'

In [30]:
onehot_repr[1]

[3519, 2843, 3717, 441, 4185, 332, 4091, 3438, 835, 3828]

Padding

In [31]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[3422 1025  474 ...    0    0    0]
 [3519 2843 3717 ...    0    0    0]
 [1539 1338 3438 ...    0    0    0]
 ...
 [1361 2467  413 ...    0    0    0]
 [ 411 1250 3355 ...    0    0    0]
 [1539  806 1070 ...    0    0    0]]


Defining the Model

In [32]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(50))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 50)                18200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 218251 (852.54 KB)
Trainable params: 218251 (852.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [33]:
len(embedded_docs),y.shape

(44898, (44898,))

In [34]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [35]:

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [36]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=7,batch_size=64)

Epoch 1/7
471/471 [==============================] - 14s 19ms/step - loss: 0.2278 - accuracy: 0.9030 - val_loss: 0.1679 - val_accuracy: 0.9328
Epoch 2/7
471/471 [==============================] - 15s 32ms/step - loss: 0.1345 - accuracy: 0.9487 - val_loss: 0.2049 - val_accuracy: 0.9241
Epoch 3/7
471/471 [==============================] - 15s 32ms/step - loss: 0.1198 - accuracy: 0.9535 - val_loss: 0.1661 - val_accuracy: 0.9346
Epoch 4/7
471/471 [==============================] - 13s 28ms/step - loss: 0.1051 - accuracy: 0.9597 - val_loss: 0.1808 - val_accuracy: 0.9356
Epoch 5/7
471/471 [==============================] - 13s 27ms/step - loss: 0.0956 - accuracy: 0.9636 - val_loss: 0.1890 - val_accuracy: 0.9304
Epoch 6/7
471/471 [==============================] - 10s 21ms/step - loss: 0.0832 - accuracy: 0.9671 - val_loss: 0.2110 - val_accuracy: 0.9322
Epoch 7/7
471/471 [==============================] - 10s 20ms/step - loss: 0.0692 - accuracy: 0.9724 - val_loss: 0.2549 - val_accuracy: 0.9323

In [37]:
y_pred=model.predict(X_test)

464/464 [==============================] - 3s 5ms/step


In [38]:
y_pred=np.where(y_pred > 0.5, 1,0)

Evaluatig the Model

In [39]:
from sklearn.metrics import confusion_matrix

In [40]:
confusion_matrix(y_test,y_pred)

array([[7274,  519],
       [ 484, 6540]], dtype=int64)

In [41]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9323074846460148

In [42]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.93      0.94      7793
           1       0.93      0.93      0.93      7024

    accuracy                           0.93     14817
   macro avg       0.93      0.93      0.93     14817
weighted avg       0.93      0.93      0.93     14817

